In [1]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F
import os
import pandas as pd

In [2]:
#CONFIGURAÇÕES SPARK
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    .config("spark.executor.memory", "16g")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


24/10/06 18:02:41 WARN Utils: Your hostname, DESKTOP-CV9TLA8 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/06 18:02:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hacs/.cache/pypoetry/virtualenvs/pyspark-delta-gPM1PAZH-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hacs/.ivy2/cache
The jars for the packages stored in: /home/hacs/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fd81439c-1097-4e7f-b076-8478e358f3c5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 148ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [3]:
#CONFIGURAÇÕES CLIENT DO MINIO
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [4]:
bucket_name = 'bronze'
objects=client.list_objects(bucket_name, prefix='CAPES/', recursive=True)
csvs = []
for object in objects:
    csvs.append(object.object_name)
    
name_csvs = []
for csv in csvs:
    name_csvs.append(csv.split('/'))

##### LISTANDO OS DATAFRAMES PRESENTES NO BUCKET RAW DO MinIO
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1

In [6]:
def unionDf(dfstart, dfend):
    df_one = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[dfstart]}')
    for i in range(dfstart+1, dfend+1):
        df_two = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[dfend]}')
        df_one = df_one.unionByName(df_two, allowMissingColumns=True)
    return df_one


In [7]:
def clearOnlyParquets(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.endswith(".parquet"):
                os.remove(os.path.join(root, file))

In [8]:
def toParquet(df, nameDf):
    df.write.mode('overwrite').parquet(nameDf)
    clearOnlyParquets(nameDf)

In [37]:
projetos = unionDf(49,50)

In [38]:
projetos = projetos.drop('AN_BASE',
                        'ID_ADD_FOTO_PROGRAMA_IES',
                        'ID_ADD_FOTO_PROGRAMA', 'ID_ADD_CONTEXTO',
                         'IN_BOLSA', 'IN_OUTRO_AUXILIO'
                        )

In [39]:
projetos = projetos.dropDuplicates()
projetos.count()

5003

In [40]:
toParquet(projetos, name_csvs[49][1])

In [13]:
programas = unionDf(47,48)
programas = programas.drop('AN_BASE',
                           'ID_ADD_FOTO_PROGRAMA_IES',
                           'ID_ADD_FOTO_PROGRAMA',
                           'IN_REDE'
                          )
programas = programas.dropDuplicates()

In [14]:
programas = programas.drop('NM_GRAU_PROGRAMA')
programas = programas.dropDuplicates()

In [15]:
toParquet(programas, name_csvs[47][1])

24/10/06 18:02:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [16]:
producao = unionDf(36,46)

In [17]:
producao = producao.drop('AN_BASE')

In [18]:
producao = producao.dropDuplicates()

In [19]:
toParquet(producao, name_csvs[36][1])

24/10/06 18:02:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/10/06 18:02:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/10/06 18:02:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/10/06 18:02:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/06 18:02:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/10/06 18:02:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/10/06 18:02:53 WARN MemoryManager: Total allocation exceeds 95.0

In [20]:
membros = unionDf(34,35)

In [21]:
membros =  membros.drop('AN_BASE', 'ID_ADD_FOTO_PROGRAMA', 'ID_ADD_FOTO_PROGRAMA_IES',
                        'ID_ADD_MEMBRO_PROJETO', 'ID_ADD_PARTICIPANTE_EXTERNO','ID_ADD_DISCENTE',
                        'ID_ADD_DOCENTE','ID_ADD_CONTEXTO')

In [22]:
membros = membros.dropDuplicates()

In [23]:
toParquet(membros, name_csvs[34][1])

24/10/06 18:02:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/10/06 18:02:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/10/06 18:02:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/10/06 18:02:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/06 18:02:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/10/06 18:02:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/06 18:02:57 WARN MemoryManager: Total allocation exceeds 95.

In [24]:
financiadores = unionDf(32,33)

In [25]:
financiadores = financiadores.drop('AN_BASE',
                   'ID_ADD_FOTO_PROGRAMA',
                   'ID_ADD_FOTO_PROGRAMA_IES',
                   'ID_ADD_CONTEXTO')
financiadores = financiadores.dropDuplicates()

In [26]:
toParquet(financiadores, name_csvs[32][1])

24/10/06 18:02:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/10/06 18:02:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/10/06 18:02:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/10/06 18:02:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/06 18:02:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/10/06 18:02:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/06 18:02:58 WARN MemoryManager: Total allocation exceeds 95.

In [27]:
autores = unionDf(0,12)

In [28]:
autores = autores.dropDuplicates()

In [29]:
toParquet(autores, name_csvs[0][1])

In [30]:
catalogos = unionDf(13,14)

In [31]:
catalogos = catalogos.drop('AN_BASE')
catalogos = catalogos.dropDuplicates()

In [32]:
toParquet(catalogos, name_csvs[13][1])

24/10/06 18:03:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [33]:
discentes = unionDf(28,29)

In [34]:
discentes = discentes.drop('AN_BASE', 'ID_ADD_FOTO_PROGRAMA', 'ID_ADD_FOTO_PROGRAMA_IES')

In [35]:
discentes = discentes.dropDuplicates()

In [36]:
toParquet(discentes,name_csvs[28][1])

In [41]:
docentes = unionDf(30,31)

In [43]:
docentes = docentes.drop('AN_BASE', 'ID_ADD_FOTO_PROGRAMA', 'ID_ADD_FOTO_PROGRAMA_IES')
docentes.count()

4410

In [44]:
docentes = docentes.dropDuplicates()
docentes.count()

2851

In [45]:
toParquet(docentes,name_csvs[30][1])

In [46]:
cursos = unionDf(15,16)

In [48]:
cursos = cursos.drop('AN_BASE', 'ID_ADD_FOTO_PROGRAMA_IES', 'ID_ADD_FOTO_PROGRAMA')
cursos.count()

260

In [49]:
cursos = cursos.dropDuplicates()

In [51]:
toParquet(cursos, name_csvs[15][1])